In [8]:
from elasticsearch import Elasticsearch
import pandas as pd
import json
import plotly.express as px

ModuleNotFoundError: No module named 'plotly'

In [9]:
!pip install plotly


     |████████████████████████████████| 12.9MB 73kB/s  eta 0:00:01
ERROR: Could not install packages due to an EnvironmentError: [Errno 28] No space left on device

You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
es = Elasticsearch(hosts='localhost:9200')

In [4]:
def get_counts(es, index='hospital-index'):
    # results are paginated, so we need a while loop
    # after the initial search until we collect all the results.
    counts = []
    query = json.dumps({
        "aggs": {
            "buckets": {
                "composite": {
                    "sources": [
                        {
                            "state": {
                                "terms": {
                                    "field": "state"
                                }
                            }
                        },
                        {
                            "measure_id": {
                                "terms": {
                                    "field": "measure_id"
                                }
                            }
                        }
                    ]
                }
            }
        }
    })
    res = es.search(index=index, body=query)
    counts += res['aggregations']['buckets']['buckets']
    while 'after_key' in res['aggregations']['buckets']:
        after_key = res['aggregations']['buckets']['after_key']
        query = json.dumps({
            "aggs": {
                "buckets": {
                    "composite": {
                        "sources": [
                            {
                                "state": {
                                    "terms": {
                                        "field": "state"
                                    }
                                }
                            },
                            {
                                "measure_id": {
                                    "terms": {
                                        "field": "measure_id"
                                    }
                                }
                            }
                        ],
                        "after": after_key
                    }
                }
            }
        })
        res = es.search(index=index, body=query)
        counts += res['aggregations']['buckets']['buckets']
    return counts




In [5]:
counts = get_counts(es)

In [6]:
counts_df = pd.DataFrame([{'state': item['key']['state'],
                           'measure_id': item['key']['measure_id'],
                           'counts': item['doc_count']}
                          for item in counts])

In [7]:
counts_df

,state,measure_id,counts
0,AK,HAI_1_CILOWER,23
1,AK,HAI_1_CIUPPER,23
2,AK,HAI_1_DOPC,23
3,AK,HAI_1_ELIGCASES,23
4,AK,HAI_1_NUMERATOR,23
...,...,...,...
2011,WY,HAI_6_CIUPPER,29
2012,WY,HAI_6_DOPC,29
2013,WY,HAI_6_ELIGCASES,29
2014,WY,HAI_6_NUMERATOR,29


In [ ]:
p1 = plt.bar(counts_df['state'], menMeans, width, yerr=menStd)
p2 = plt.bar(ind, womenMeans, width,
             bottom=menMeans, yerr=womenStd)

plt.ylabel('Scores')
plt.title('Scores by group and gender')
plt.xticks(ind, ('G1', 'G2', 'G3', 'G4', 'G5'))
plt.yticks(np.arange(0, 81, 10))
plt.legend((p1[0], p2[0]), ('Men', 'Women'))

plt.show()